In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import keras_tuner as kt
from scipy import signal
import scipy.io as sio
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout, Lambda, AveragePooling2D, Softmax, ReLU, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import time
from math import floor,inf
from sklearn.utils import shuffle

In [2]:
data_folder = 'BBCI competition IV 2a'  ###location of .mat files for each subject, change file path if running notebook from...
                                        ###...different directory to data folder
subject_files = os.listdir(data_folder)

In [3]:
subject_files

['A01E.mat',
 'A01T.mat',
 'A02E.mat',
 'A02T.mat',
 'A03E.mat',
 'A03T.mat',
 'A04E.mat',
 'A04T.mat',
 'A05E.mat',
 'A05T.mat',
 'A06E.mat',
 'A06T.mat',
 'A07E.mat',
 'A07T.mat',
 'A08E.mat',
 'A08T.mat',
 'A09E.mat',
 'A09T.mat']

In [14]:
####Create dataset of only the 4s in the trials where MI was being performed.
def create_4sec_data(subject,dataset):
    data_folder = 'BBCI competition IV 2a'
    file = 'A0' + str(subject) + dataset[0] + '.mat'
    file_path = os.path.join(data_folder, file )
    print(file_path)
    data = sio.loadmat(file_path)['data']
    x=[]
    y=[]
    if file == 'A04T.mat':
        lower_lim = 1
        upper_lim = 7
    else:
        lower_lim = 3
        upper_lim = 9
    for s in range(lower_lim, upper_lim):
        eeg = pd.DataFrame(np.transpose(data[0,s][0,0][0])[:22])
        cues = data[0,s][0,0][1]
        mi = data[0,s][0,0][2]
        for i in range(48):
            start = cues[i][0] + 500
            end = start + 1000
            mi_data = eeg.iloc[:,start:end]
            x.append(np.array(mi_data))
            if mi[i][0] == 3:
                y.append(0)
            elif mi[i][0] == 4:
                y.append(3)
            else:
                y.append(mi[i][0])
            
    return np.array(x), np.array(y)

In [15]:
###Function to segment and filter 4s data into 2s segments
def filt_and_seg(x_4sec, y_4sec, samples, overlap):
    unique = floor((1-overlap)*samples)
    b,a = signal.butter(2, Wn = [48,52], btype = 'bandstop', fs = 250)
    d,c = signal.butter(2, Wn = [4,38], btype = 'bandpass', fs = 250)
    x=[]
    y=[]
    count = 0
    for trial in x_4sec:
        data = trial
        label = y_4sec[count]
        for div in range(int(data.size//(data.shape[0]*unique))):
            if div*unique +samples <= data.size/data.shape[0]:
                array = np.ndarray((22,samples))
                for i in range(22):
                    notch_filt = signal.filtfilt(b,a, data[i][div*unique:div*unique + samples], padtype='even')
                    bp_filt = signal.filtfilt(d,c, notch_filt, padtype='even')
                    array[i] = bp_filt
                x.append(array)
                y.append(label)
        count+=1
    x=np.array(x)
    x=x.reshape(x.shape[0],x.shape[1],x.shape[2],1)
    x, y = shuffle(x,y, random_state=42)
    return x,y

In [ ]:
#######TUNING##########

In [34]:
subject = 9     #####no loop was used, manually change from 1 to 9 to tune models for each subject
x1T, y1T= create_4sec_data(subject,'T')
x1E, y1E= create_4sec_data(subject,'E')
x1_train, y1_train = filt_and_seg(x1T,y1T, 500, 0.90)
x1_val, y1_val = filt_and_seg(x1E,y1E, 500, 0.90)
y1_cat_train = to_categorical(y1_train)
y1_cat_val = to_categorical(y1_val)

BBCI competition IV 2a\A09T.mat
BBCI competition IV 2a\A09E.mat


In [35]:
####  MODEL BUILDER with hyperparameter search spaces ####
def build_model(hp):
    model = Sequential()
    
    hp_filters = hp.Int('Filters', min_value=10,max_value=160,step=10)
    hp_kern_len = hp.Int('Kernel1 Length', min_value=10, max_value=250, step=10)
    model.add(Conv2D(filters=hp_filters, kernel_size=(1,hp_kern_len), input_shape=(x1_train.shape[1],x1_train.shape[2],1))) #, activation = 'relu'))
    model.add(Conv2D(filters=hp_filters, kernel_size=(22,1), activation = 'elu'))
    hp_momentum = hp.Float('Momentum', min_value=0.05, max_value=0.95, step=0.05)
    model.add(BatchNormalization(momentum=hp_momentum, epsilon = 1e-05))
    model.add(Lambda(lambda x: x**2))
    hp_pool_len = hp.Int('Pooling Length', min_value=10, max_value=250, step=10)
    hp_strides = hp.Int('Pooling Strides', min_value = 5, max_value=50, step=5)
    model.add(AveragePooling2D(pool_size=(1,hp_pool_len),strides=hp_strides))
    
    model.add(Lambda(lambda x: tf.math.log(tf.clip_by_value(x, 1e-6, inf))))
    hp_dropout = hp.Float('Dropout Rate', min_value=0.1, max_value=0.9, step = 0.1)
    model.add(Dropout(hp_dropout))
    
    '''model.add(Conv2D(filters=4, kernel_size=(1,69))) #, activation = 'relu'))       
    
    model.add(Softmax())
    
    model.add(Lambda(lambda x: tf.squeeze(x, [1,2])))'''
    
    model.add(Flatten())
    
    model.add(Dense(4, activation = 'softmax'))
    
    loss_fn = 'categorical_crossentropy'
    
    hp_learn_rate = hp.Choice('Learning Rate', values=[1e-1,1e-2,1e-3,1e-4,1e-5,1e-6])
    
    
    adam = tf.keras.optimizers.Adam(learning_rate=hp_learn_rate)
    model.compile(loss = loss_fn, optimizer=adam, metrics=['accuracy'], )  # other metrics at keras.io/metrics
    
    return model

In [40]:
####Initialise tuner
tuner = kt.Hyperband(build_model,
                     objective = 'val_accuracy', 
                     max_epochs = 500,
                     factor = 3,
                     directory=os.getcwd(),
                     project_name = 'TunedCNN-4class MI- BCI Comp S' + str(subject)) ##Folder name to save tuning results

INFO:tensorflow:Reloading Oracle from existing project C:\Users\Anthony\OneDrive - University of Witwatersrand\2020 MASTERS\GUI\TunedCNN-4class MI- BCI Comp S9\oracle.json
INFO:tensorflow:Reloading Tuner from C:\Users\Anthony\OneDrive - University of Witwatersrand\2020 MASTERS\GUI\TunedCNN-4class MI- BCI Comp S9\tuner0.json


In [41]:
####Summary of hyperparameter search spaces
tuner.search_space_summary()

Search space summary
Default search space size: 7
Filters (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 160, 'step': 10, 'sampling': None}
Kernel1 Length (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 250, 'step': 10, 'sampling': None}
Momentum (Float)
{'default': 0.05, 'conditions': [], 'min_value': 0.05, 'max_value': 0.95, 'step': 0.05, 'sampling': None}
Pooling Length (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 250, 'step': 10, 'sampling': None}
Pooling Strides (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 50, 'step': 5, 'sampling': None}
Dropout Rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.9, 'step': 0.1, 'sampling': None}
Learning Rate (Choice)
{'default': 0.1, 'conditions': [], 'values': [0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06], 'ordered': True}


In [42]:
early_stop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

In [43]:
####Tuning
tuner.search(x1_train,y1_cat_train, epochs = 300, validation_data=(x1_val,y1_cat_val), callbacks = [early_stop], verbose=2)

Trial 726 Complete [00h 09m 04s]
val_accuracy: 0.5950126051902771

Best val_accuracy So Far: 0.7490530014038086
Total elapsed time: 03h 30m 18s
INFO:tensorflow:Oracle triggered exit


In [44]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 491, 50)       550       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 491, 50)        55050     
_________________________________________________________________
batch_normalization (BatchNo (None, 1, 491, 50)        200       
_________________________________________________________________
lambda (Lambda)              (None, 1, 491, 50)        0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 1, 9, 50)          0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 1, 9, 50)          0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 9, 50)          0

In [45]:
#####Examples to get best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=2)[0]

In [46]:
best_hps['Learning Rate']

0.001

In [47]:
best_hps['Momentum']

0.1

In [49]:
###Tuning results showing hyperparameters and accuracies for 10 best models
tuner.results_summary()

Results summary
Results in C:\Users\Anthony\OneDrive - University of Witwatersrand\2020 MASTERS\GUI\TunedCNN-4class MI- BCI Comp S9
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
Filters: 50
Kernel1 Length: 10
Momentum: 0.1
Pooling Length: 90
Pooling Strides: 45
Dropout Rate: 0.7000000000000001
Learning Rate: 0.001
tuner/epochs: 167
tuner/initial_epoch: 56
tuner/bracket: 5
tuner/round: 4
tuner/trial_id: 42f568271255d80c4ae4f58fdcf59a7e
Score: 0.7490530014038086
Trial summary
Hyperparameters:
Filters: 50
Kernel1 Length: 10
Momentum: 0.1
Pooling Length: 90
Pooling Strides: 45
Dropout Rate: 0.7000000000000001
Learning Rate: 0.001
tuner/epochs: 56
tuner/initial_epoch: 19
tuner/bracket: 5
tuner/round: 3
tuner/trial_id: dab9ce84f1f3dc228b116f255127aa97
Score: 0.747474730014801
Trial summary
Hyperparameters:
Filters: 50
Kernel1 Length: 10
Momentum: 0.1
Pooling Length: 90
Pooling Strides: 45
Dropout Rate: 0.7000000000000001
Learning Rate: 